In [6]:
from google.colab import files
import pandas as pd
import zipfile
import io

# 1. FAZER UPLOAD DO ARQUIVO ZIP (Passo crucial)
print("Por favor, faça upload do arquivo ZIP contendo os datasets:")
uploaded = files.upload()

# 2. EXTRAIR OS ARQUIVOS DO ZIP
zip_filename = list(uploaded.keys())[0]  # Pega o nome do primeiro arquivo enviado

with zipfile.ZipFile(io.BytesIO(uploaded[zip_filename]), 'r') as zip_ref:
    zip_ref.extractall('.')  # Extrai todos os arquivos para a pasta atual
    print("Arquivos extraídos:")
    print(zip_ref.namelist())  # Mostra lista de arquivos no ZIP

# 3. VERIFICAR NOMES EXATOS DOS ARQUIVOS (Importante!)
# Os nomes podem variar dependendo da fonte do dataset
print("\nArquivos disponíveis após extração:")
!ls -l

# 4. CARREGAR OS DATASETS (com nomes corrigidos)
try:
    orders = pd.read_csv('olist_orders_dataset.csv')  # Nome padrão
except FileNotFoundError:
    # Tentar variações comuns de nomes
    orders = pd.read_csv('orders.csv')  # Nome alternativo

# Repetir para outros arquivos
order_items = pd.read_csv('olist_order_items_dataset.csv')  # ou 'order_items.csv'
products = pd.read_csv('olist_products_dataset.csv')  # ou 'products.csv'
customers = pd.read_csv('olist_customers_dataset.csv')  # ou 'customers.csv'

# 5. VERIFICAÇÃO DE CARREGAMENTO
print("\nPrimeiras linhas de cada dataset:")
print("Pedidos:", orders.shape, orders.columns.tolist())
print("Itens:", order_items.shape, order_items.columns.tolist())
print("Produtos:", products.shape, products.columns.tolist())
print("Clientes:", customers.shape, customers.columns.tolist())

Por favor, faça upload do arquivo ZIP contendo os datasets:


Saving archive.zip to archive (5).zip
Arquivos extraídos:
['olist_customers_dataset.csv', 'olist_geolocation_dataset.csv', 'olist_order_items_dataset.csv', 'olist_order_payments_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_orders_dataset.csv', 'olist_products_dataset.csv', 'olist_sellers_dataset.csv', 'product_category_name_translation.csv']

Arquivos disponíveis após extração:
total 385284
-rw-r--r-- 1 root root 44717580 Mar 27 18:42 'archive (1).zip'
-rw-r--r-- 1 root root 44717580 Mar 27 18:45 'archive (2).zip'
-rw-r--r-- 1 root root 44717580 Mar 27 18:48 'archive (3).zip'
-rw-r--r-- 1 root root 44717580 Mar 27 18:54 'archive (4).zip'
-rw-r--r-- 1 root root 44717580 Mar 27 19:02 'archive (5).zip'
-rw-r--r-- 1 root root 44717580 Mar 27 18:40  archive.zip
-rw-r--r-- 1 root root  9033957 Mar 27 19:02  olist_customers_dataset.csv
-rw-r--r-- 1 root root 61273883 Mar 27 19:02  olist_geolocation_dataset.csv
-rw-r--r-- 1 root root 15438671 Mar 27 19:02  olist_order_items_dataset.

In [7]:
# 1.2.1 - JUNÇÃO DAS TABELAS PRINCIPAIS
# --------------------------------------
# Unir itens de pedido com informações de produtos
order_items_with_products = order_items.merge(
    products[['product_id', 'product_category_name']],  # Selecionar apenas colunas necessárias
    on='product_id',
    how='left'
)

# Verificar resultado
print("Itens de pedido com categorias:")
print(order_items_with_products.head(3))
print(f"\nTotal de linhas: {len(order_items_with_products)}")
print(f"Colunas disponíveis: {order_items_with_products.columns.tolist()}")

# 1.2.2 - ADICIONAR INFORMAÇÕES DE PEDIDOS
# ----------------------------------------
full_data = order_items_with_products.merge(
    orders[['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp']],
    on='order_id',
    how='left'
)

# 1.2.3 - ADICIONAR DADOS DE CLIENTES
# -----------------------------------
full_data = full_data.merge(
    customers[['customer_id', 'customer_state']],
    on='customer_id',
    how='left'
)

# Verificação final
print("\nDataset completo:")
print(full_data.head(3))
print(f"\nTotal de linhas final: {len(full_data)}")
print(f"Colunas disponíveis: {full_data.columns.tolist()}")

# 1.2.4 - TRATAMENTO DE VALORES AUSENTES
# --------------------------------------
# Preencher categorias não informadas
full_data['product_category_name'] = full_data['product_category_name'].fillna('nao_informado')

# Verificar nulos
print("\nValores nulos por coluna:")
print(full_data.isnull().sum())

Itens de pedido com categorias:
                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36   
2  c777355d18b72b67abbeef9df44fd0fd  5b51032eddd242adc84c38acab88f23d   

   shipping_limit_date  price  freight_value product_category_name  
0  2017-09-19 09:45:35   58.9          13.29            cool_stuff  
1  2017-05-03 11:05:13  239.9          19.93              pet_shop  
2  2018-01-18 14:48:30  199.0          17.87      moveis_decoracao  

Total de linhas: 112650
Colunas disponíveis: ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value', 'product_category_name']

Data

In [8]:
# Converter datas
full_data['order_purchase_timestamp'] = pd.to_datetime(full_data['order_purchase_timestamp'])

# Criar colunas derivadas
full_data['purchase_month'] = full_data['order_purchase_timestamp'].dt.to_period('M')
full_data['total_price'] = full_data['price'] + full_data['freight_value']

# Tratar categorias ausentes
full_data['product_category_name'] = full_data['product_category_name'].fillna('nao_informado')

# Verificar resultado
print(full_data.head(3))

                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36   
2  c777355d18b72b67abbeef9df44fd0fd  5b51032eddd242adc84c38acab88f23d   

   shipping_limit_date  price  freight_value product_category_name  \
0  2017-09-19 09:45:35   58.9          13.29            cool_stuff   
1  2017-05-03 11:05:13  239.9          19.93              pet_shop   
2  2018-01-18 14:48:30  199.0          17.87      moveis_decoracao   

                        customer_id order_status order_purchase_timestamp  \
0  3ce436f183e68e07877b285a838db11a    delivered      2017-09-13 08:59:02   
1  f6dd3ec061db4e3987629fe6b26e5cce    delivered      2

In [9]:
# Exportar dados unificados
full_data.to_csv('olist_full_data.csv', index=False)

# Baixar arquivo
files.download('olist_full_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>